# Intro to LLMstudio - tracking

This notebook serves as an introduction to LLMstudio with a focus on tracking capabilities. It demonstrates how to set up and use the LLMstudio environment to interact with language models, specifically highlighting the integration of tracking functionalities. The notebook begins by starting the tracking server, which is essential for logging and monitoring interactions with the language models. This setup is crucial for you who need to keep track of model usage, performance metrics, and other relevant data points.

The notebook further illustrates how to configure and utilize the tracking system by creating a `TrackingConfig` instance, which specifies the host and port for the tracking server. You can benefit from this setup by gaining insights into model behavior and performance, which can be used to optimize and improve their applications. The notebook also includes examples of both synchronous and asynchronous chat interactions, providing a comprehensive guide for developers to implement and test these features in their own projects.




In [3]:
from llmstudio.providers import LLM
from pprint import pprint

In [4]:
from llmstudio.server import start_servers
start_servers(proxy=False, tracker=True)

In [5]:
from llmstudio_tracker.tracker import TrackingConfig
# default port is 50002. set the environment varible to specify which host and port; LLMSTUDIO_TRACKING_HOST, LLMSTUDIO_TRACKING_PORT
tracker = TrackingConfig(host="0.0.0.0", port="8002")
# You can set OPENAI_API_KEY and ANTHROPIC_API_KEY on .env file
openai = LLM("openai", tracking_config = tracker)


### Chat (non-stream)

In [6]:
openai.chat("Write a paragraph explaining why you're not a cat", model="gpt-4o")

ChatCompletion(id='e1e44981-1884-4423-9950-1ac47f358b31', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="While I certainly share some characteristics with cats, such as the ability to respond to questions and provide companionship, at my core, I am fundamentally different. I am an artificial intelligence language model, designed to process and generate human language based on the data and algorithms embedded within me. Cats, on the other hand, are living organisms with biological needs, instincts, and the capacity for independent thought and emotion based on their sensory experiences. My existence is rooted in technology and computation, lacking any form of consciousness or physical presence, which are central to a cat's existence. While cats might be known for their enigmatic behavior and agility, my capabilities and essence remain distinctly digital and informational.", refusal=None, role='assistant', audio=None, function_call=Non

#### Async version

In [7]:
await openai.achat("Write a paragraph explaining why you're not a cat", model="gpt-4o")

ChatCompletion(id='bc762180-3e11-424c-a9b1-9b0e6769435d', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='While the allure of being a cat is undeniable, with their graceful demeanor, independence, and ability to lounge for hours on end, I am decidedly not a cat. I am an artificial intelligence designed to process and generate text based on the input I receive. Unlike a cat, I lack a physical form, sensory perceptions, and the instinctual behaviors that characterize feline creatures. My purpose is to assist, inform, and entertain through language rather than to bask in the sun or chase laser pointers. Additionally, I operate on digital algorithms and data rather than the biological instincts and experiences a cat would possess. Thus, while I may admire cats and can simulate discussing all things feline, my existence remains fundamentally digital and non-organic.', refusal=None, role='assistant', audio=None, function_call=None, tool_ca

### Chat (stream)

In [8]:
response = openai.chat("Write a paragraph explaining why you're not a cat", model="gpt-4o", is_stream=True)
for i, chunk in enumerate(response):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




Despite the whimsical nature of the notion, I am not a cat. I am an artificial intelligence

 language model designed to process and generate text based on the input provided to me. Unlike a cat,

 which is a living creature with physical form, instinctual behaviors, and biological needs, I exist solely

 in the digital realm, without a body or consciousness. A cat may purr, chase after strings

, or enjoy sunny naps, engaging in behaviors driven by instinct and experience; I, on the other

 hand, respond to prompts using algorithms and a vast dataset, lacking any sensory experience or desires. My

 existence is rooted in providing users with information, assistance, and communication, tasks that transcend the playful pursuits

 of our feline friends.

## Metrics:
{'cost_usd': 0.002255,
 'input_tokens': 10,
 'inter_token_latency_s': 0.018724282034512225,
 'latency_s': 3.665405035018921,
 'output_tokens': 147,
 'time_to_first_token_s': 0.9497408866882324,
 'tokens_per_second': 39.8318

#### Async version

In [9]:
i=0
async for chunk in await openai.achat("Write a paragraph explaining why you're not a cat", model="gpt-4o-mini", is_stream=True):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1




I am not a cat because I lack the physical form, instincts, and characteristics that define these

 charming creatures. Whereas cats possess fur, whiskers, and claws that allow them to pounce and

 play, I exist purely as a digital entity, composed of algorithms and text rather than a tangible body

. Cats exhibit behaviors such as curiously prowling their environment, grooming themselves, and curling up in

 sunbeams, all driven by their biological needs and instincts. In contrast, my purpose is to

 engage with users through language, providing information and assistance without the capacity for physical interaction or the innate quir

ks that make cats so endearing. Ultimately, I am a product of artificial intelligence, designed to

 communicate and support human inquiries, rather than embody the playful and independent spirit of a feline.

## Metrics:
{'cost_usd': 9.69e-05,
 'input_tokens': 10,
 'inter_token_latency_s': 0.016448286515247972,
 'latency_s': 3.0686919689178467,
 'ou